<a href="https://colab.research.google.com/github/alvarofpinheiro/pifwia_pso/blob/main/PIFWIA_PSO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Particle Swarm Optimization (PSO) algoritmo de Inteligência Computacional utilizado para otimização utilizando o conceito de enxame de partículas, sendo um método computacional que otimiza um problema tentando melhorar iterativamente uma solução candidata em relação a uma determinada medida de qualidade. Ele resolve um problema tendo uma população de soluções candidatas, aqui chamadas de partículas, e movendo essas partículas no espaço de busca de acordo com uma fórmula matemática simples sobre a posição e a velocidade da partícula. O movimento de cada partícula é influenciado por sua melhor posição local conhecida, mas também é guiado para as melhores posições conhecidas no espaço de busca, que são atualizadas à medida que as melhores posições são encontradas por outras partículas. Espera-se que isso mova o enxame em direção às melhores soluções.

Bonyadi, M. R.; Michalewicz, Z. Particle swarm optimization for single objective continuous space problems: a review. Evolutionary Computation. 25 (1): 1–54. PMID 26953883. S2CID 8783143. https://doi.org/10.1162/EVCO_r_00180, 2017.

Pseudo-código:

```
do
  for each particle
    calculate fitness value
    if new fitness value is better than the personal best
      set current fitness value as a new personal best
  end
  update global best
  for each particle
    calculate particle velocity
    update particle position
  end
while maximum iterations is not attained
```

## Sphere Function

$F_{Sphere} = \sum_{i=1}^{D} x_i^2$

## Cálculo da Velocidade
$ {v}^{t+1}_{i,d}= {\omega}{v}^{t}_{i,d}+ {c_1}{rand_1} \otimes ({pbest}^{t}_{i,d}-{x}_{t}^{p}) + {c_2}{rand_2}\otimes ({gbest}^{t}_{i,d}-{x}^{t}_{i,d})$

## Atualização da Posição da Partícula
$\mathbf{x}^{t+1}_{p}=\mathbf{x}^{t}_{p}+ \mathbf{v}^{t+1}_p$

In [ ]:
#Bibliotecas
import random
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
#Particula
#Indivíduo da população ou enxame.
#Cada partícula possui uma determinada posição no espaço de busca do problema e esta posição representa uma solução em potencial para o problema tratado.

class Particula:
  def __init__(self):
        self.lista_posicao = []
        self.lista_velocidade = []
        self.fitness = np.inf
        self.fitness_pbest = np.inf
        self.lista_posicao_pbest = []        

In [ ]:
#Atributos PSO
#Coeficiente de inercia (𝑤): determina o quanto a velocidade anterior influencia na velocidade atual;
#Fator de individualidade (𝑐1): Influencia na atração que a partícula tem em direção à melhor posição já encontrada por ela mesma (𝑃𝑏𝑒𝑠𝑡);
#Fator de sociabilidade (𝑐2): Influencia na atração que a partícula tem em direção à melhor posição já encontrada por qualquer partícula vizinha a ela (𝐿𝑏𝑒𝑠𝑡);
#Limite (bound): Determina os valores maximos e minimos do espaço de busca;
#Velocidade Máxima
#Dimensão: Determina a quantidade de dimensões do problema
#Quantidade de Particulas
#Quantidade de Iterações
#Swarm: População

w = 0.8 #coeficiente de inércia
c1 = 2.05 #fator de individualidade
c2 = 2.05 #fator de sociabilidade
bound = [-100,100]
velocidade_max = [-6,6]
dimensao = 30 #2
numero_particulas = 30 #2
qtd_iteracoes = 10000 #2
swarm = []

In [ ]:
#Inicialização do enxame

for i in range(numero_particulas):
  p = Particula()
  for i in range (dimensao):
    p.lista_posicao.append(random.uniform(bound[0], bound[1]))
    #print(p.lista_posicao)
    p.lista_velocidade.append(random.uniform(velocidade_max[0], velocidade_max[1]))
    #print(p.lista_velocidade)
  swarm.append(p)
  #print(p)
  #print(swarm)

In [ ]:
#Exibir o enxame

#print(numero_particulas)
for i in range (numero_particulas): #(numero_particulas):
    #print(i)
    #print(swarm[i])
    p = swarm[i]
    #print(p)
    #print(p.lista_posicao)
    d1,d2,d3,d4,d5,d6,d7,d8,d9,d10,d11,d12,d13,d14,d15,d16,d17,d18,d19,d20,d21,d22,d23,d24,d25,d26,d27,d28,d29,d30 = zip(p.lista_posicao)
    plt.plot(d1,d2,d3,d4,d5,d6,d7,d8,d9,d10,d11,d12,d13,d14,d15,d16,d17,d18,d19,d20,d21,d22,d23,d24,d25,d26,d27,d28,d29,d30, marker='o')
plt.plot(0,0, marker='*')
plt.axis([-100, 100, -100, 100])
plt.show()

In [ ]:
#Função fitness

def funcao_fitness(lista_solucao):
  total = 0
  for i in range (dimensao):
      total += lista_solucao[i]**2
  return total

In [ ]:
#Cálculo Fitness

def calculo_fitness(particula):
  particula.fitness = funcao_fitness(particula.lista_posicao)
  if (particula.fitness < particula.fitness_pbest):
    particula.fitness_pbest = particula.fitness
    particula.lista_posicao_pbest = list (particula.lista_posicao)

In [ ]:
#Cálculo da velocidade

def atualizacao_velocidade_global(particula, lista_gbest):
  for i in range(dimensao):
    e1 = random.random()
    e2 = random.random()
    velocidade_cognitiva = c1*e1* (particula.lista_posicao_pbest[i] - particula.lista_posicao[i])
    velocidade_social = c2*e2* (lista_gbest[i] - particula.lista_posicao[i])
    v = w * particula.lista_velocidade[i] + velocidade_cognitiva + velocidade_social
    if v>bound[1]:
      v = bound[1]
    elif v<bound[0]:
      v = bound[0]
    particula.lista_velocidade[i] = v

In [ ]:
#Atualização da posição da partícula

def atualiza_posicao(particula, bound):
  for i in range(dimensao):
      novo_valor = particula.lista_posicao[i] + particula.lista_velocidade[i]
      if novo_valor > bound[1]:
          novo_valor =  bound[1]
      if novo_valor < bound[0]:
          novo_valor = bound[0]
      particula.lista_posicao[i] = novo_valor

In [ ]:
#PSO

fitness_gbest = float('inf')
lista_posicao_gbest = []
lista_valores_gbest = []
for i in range(qtd_iteracoes):
  for j in range(numero_particulas):
     calculo_fitness(swarm[j])
     if  swarm[j].fitness < fitness_gbest:
      fitness_gbest = swarm[j].fitness
      lista_posicao_gbest = list (swarm[j].lista_posicao)
  for j in range(numero_particulas):
    atualizacao_velocidade_global(swarm[j],lista_posicao_gbest)
    atualiza_posicao(swarm[j],bound)
  lista_valores_gbest.append(fitness_gbest)
  #EXIBE O SWARM
  for i in range(numero_particulas):
    p = swarm[i]
    d1,d2,d3,d4,d5,d6,d7,d8,d9,d10,d11,d12,d13,d14,d15,d16,d17,d18,d19,d20,d21,d22,d23,d24,d25,d26,d27,d28,d29,d30 = zip(p.lista_posicao)
    plt.plot(d1,d2,d3,d4,d5,d6,d7,d8,d9,d10,d11,d12,d13,d14,d15,d16,d17,d18,d19,d20,d21,d22,d23,d24,d25,d26,d27,d28,d29,d30, marker='o')
  #ponto ideal
  plt.plot(0,0, marker='*')
  #ponto gbest
  plt.plot(lista_posicao_gbest[0],lista_posicao_gbest[1], marker='x')
  plt.axis([-100, 100, -100, 100])
  plt.show()


In [ ]:
#Resultado

plt.plot(lista_valores_gbest)
plt.title("Convergence Curve of the PSO")
plt.xlabel("Iterations")
plt.ylabel("Best Fitness")
plt.tight_layout()

print('Valor gbest:',fitness_gbest)
